# 이기적 빅데이터 분석기사 실기

## 작업형 2유형 - 기출문제 6회(23.06.24)

In [1]:
# 학생들의 생활패턴과 신체, 거주지역 등에 대한 데이터
# 학생 530명에 대한 학습용 데이터를 이용하여 학생 구분(유치원생, 초등학생, 중학생, 고등학생, 대학생) 예측 모형을 만든 후, 
# 이를 평가용 데이터에 적용하여 얻은 230명 학생의 구분을 다음과 같은 형식의 CSV 파일로 만드시오
# 구분예측
# 중학생
# 초등학생
# 대학생
# 고등학생
# ...

### 데이터 확인

In [2]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score

In [3]:
# 데이터 불러오기
train = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/dataset-main/data_6_2_train.csv')
test = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/dataset-main/data_6_2_test.csv')
display(train.head())
display(test.head())

,일평균수면시간,일평균학습시간,일평균스마트폰사용시간,키,몸무게,체격,기초대사량,거주지역,구분
0,7.35,7.74,4.8,Tall,52.050,L,1499.4,동구,고등학생
1,7.05,8.10,1.2,Tall,51.300,M,1624.4,서구,중학생
2,7.35,7.74,3.0,Tall,51.285,L,1499.4,동구,고등학생
3,8.58,6.66,4.8,Short,25.545,XS,1249.5,서구,초등학생
4,7.05,8.10,4.8,Tall,54.150,M,1624.4,서구,대학생


,일평균수면시간,일평균학습시간,일평균스마트폰사용시간,키,몸무게,체격,기초대사량,거주지역
0,7.97,7.11,1.2,Tall,51.375,L,1749.3,북구
1,9.80,5.76,1.2,Short,28.710,XS,1749.3,북구
2,7.66,7.38,1.2,Tall,36.915,L,1624.4,남구
3,9.50,5.94,3.0,Short,24.045,XS,1624.4,동구
4,6.44,8.82,3.0,Tall,45.180,S,1499.4,서구


### EDA

In [4]:
# train데이터 타입 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   일평균수면시간      530 non-null    float64
 1   일평균학습시간      530 non-null    float64
 2   일평균스마트폰사용시간  530 non-null    float64
 3   키            530 non-null    object 
 4   몸무게          530 non-null    float64
 5   체격           530 non-null    object 
 6   기초대사량        530 non-null    float64
 7   거주지역         530 non-null    object 
 8   구분           530 non-null    object 
dtypes: float64(5), object(4)
memory usage: 37.4+ KB


In [5]:
# train데이터 기초통계 확인
train.describe(include = 'object')

,키,체격,거주지역,구분
count,530,530,530,530
unique,2,4,4,5
top,Tall,XS,남구,유치원생
freq,277,253,145,140


In [6]:
# train 데이터 결측치 확인
train.isnull().sum()

일평균수면시간        0
일평균학습시간        0
일평균스마트폰사용시간    0
키              0
몸무게            0
체격             0
기초대사량          0
거주지역           0
구분             0
dtype: int64

In [7]:
# train 데이터 타겟 값 분포 확인
train['구분'].value_counts()

구분
유치원생    140
초등학생    121
고등학생    120
대학생      78
중학생      71
Name: count, dtype: int64

In [8]:
# test 데이터 확인
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   일평균수면시간      230 non-null    float64
 1   일평균학습시간      230 non-null    float64
 2   일평균스마트폰사용시간  230 non-null    float64
 3   키            230 non-null    object 
 4   몸무게          230 non-null    float64
 5   체격           230 non-null    object 
 6   기초대사량        230 non-null    float64
 7   거주지역         230 non-null    object 
dtypes: float64(5), object(3)
memory usage: 14.5+ KB


In [9]:
# test 데이터 기초통계 확인
test.describe(include = 'object')

,키,체격,거주지역
count,230,230,230
unique,2,4,4
top,Short,XS,북구
freq,126,126,74


In [10]:
# test 데이터 결측치 확인
test.isnull().sum()

일평균수면시간        0
일평균학습시간        0
일평균스마트폰사용시간    0
키              0
몸무게            0
체격             0
기초대사량          0
거주지역           0
dtype: int64

### 데이터 전처리

In [11]:
# train 데이터의 타겟 값 처리
target = train.pop('구분')

In [12]:
# train/test 데이터 원핫 인코딩
print(train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

(530, 8) (230, 8)
(530, 15) (230, 15)


### 검증용 데이터 생성

In [13]:
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size = 0.2, random_state = 0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(424, 15) (106, 15) (424,) (106,)


### 모델링

In [14]:
# 분류 모델 생성
rf = RandomForestClassifier(random_state = 0)
rf.fit(X_train, y_train)
pred = rf.predict(X_val)

f1_score(y_val, pred, average = 'macro')

0.9317983948418732

In [15]:
# 테스트 데이터로 예측, 결과 파일 생성
pred = rf.predict(test)
result = pd.DataFrame({'pred': pred})
result.to_csv('result.csv', index = False, encoding = "utf-8-sig")

## 작업형 2유형 - 기출문제 5회(22.12.03)

In [16]:
# # used_car 데이터셋 - 중고차 관련한 정보 데이터
# 주어진 학습용 데이터를 이용하여 중고차의 판매 가격을 예측하고,
# 이를 평가용 데이터에 적용하여 얻은 예측 결과를 다음과 같은 형식의 CSV 파일로 생성하시오

### 데이터 확인

In [17]:
# 데이터 불러오기
X_test = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/dataset-main/used_car_X_test.csv')
X_train = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/dataset-main/used_car_X_train.csv')
y_train = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/dataset-main/used_car_y_train.csv')
display(X_train.head())

,id,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,1,A1,2019,Automatic,3500,Petrol,145,40.9,2.0
1,2,RS4,2020,Semi-Auto,2500,Petrol,145,28.8,2.9
2,3,A8,2019,Semi-Auto,500,Diesel,145,40.4,3.0
3,4,Q5,2019,Semi-Auto,5089,Diesel,150,38.2,2.0
4,5,A5,2020,Semi-Auto,4951,Diesel,145,51.4,2.0


In [18]:
# 각 데이터의 결측치 확인
print(f' X_train: {X_train.isnull().sum().sum()}')
print(f' X_test: {X_test.isnull().sum().sum()}')
print(f' y_train: {y_train.isnull().sum().sum()}')

 X_train: 0
 X_test: 0
 y_train: 0


### 데이터 전처리

In [19]:
# 데이터 나누기
# 범주형 데이터
X_train_word = X_train[['model', 'transmission', 'fuelType']]
X_test_word = X_test[['model', 'transmission', 'fuelType']]
print(X_train_word)

     model transmission fuelType
0       A1    Automatic   Petrol
1      RS4    Semi-Auto   Petrol
2       A8    Semi-Auto   Diesel
3       Q5    Semi-Auto   Diesel
4       A5    Semi-Auto   Diesel
...    ...          ...      ...
7463    A3       Manual   Petrol
7464    Q2       Manual   Diesel
7465    A1    Semi-Auto   Petrol
7466    Q3       Manual   Diesel
7467    Q7    Automatic   Diesel

[7468 rows x 3 columns]


In [20]:
# 수치형 데이터
X_train_num = X_train.drop(columns = ['id', 'model', 'transmission', 'fuelType'])
X_test_num = X_test.drop(columns = ['id', 'model', 'transmission', 'fuelType'])
print(X_train_num)

      year  mileage  tax   mpg  engineSize
0     2019     3500  145  40.9         2.0
1     2020     2500  145  28.8         2.9
2     2019      500  145  40.4         3.0
3     2019     5089  150  38.2         2.0
4     2020     4951  145  51.4         2.0
...    ...      ...  ...   ...         ...
7463  2016    22633   30  58.9         1.4
7464  2017    13272  145  64.2         1.6
7465  2019     5000  145  40.9         2.0
7466  2019       10  145  42.8         2.0
7467  2012    91229  300  39.2         3.0

[7468 rows x 5 columns]


### 데이터 스케일링

In [21]:
# Min-Max 스케일링 생성
scaler = MinMaxScaler()

# 선택한 특성에 MinMax 스케일러를 적용하고 데이터 변환
X_train_scale = scaler.fit_transform(X_train_num)
X_test_scale = scaler.fit_transform(X_test_num)

# 데이터프레임 설정
df_train_num = pd.DataFrame(X_train_scale, columns = X_train_num.columns)
df_test_num = pd.DataFrame(X_test_scale, columns = X_test_num.columns)

# 원핫 인코딩
df_train_word = pd.get_dummies(X_train_word)
df_test_word = pd.get_dummies(X_test_word)

In [22]:
# train 데이터프레임 컬럼목록
train_columns = set(df_train_word.columns)

# test 데이터프레임 컬럼목록
test_columns = set(df_test_word.columns)

In [23]:
# train 데이터에는 있지만 test 데이터에는 없는 컬럼 확인
missing_in_test = train_columns - test_columns

# test 데이터에는 있지만 train 데이터에는 없는 컬럼 확인
missing_in_train = test_columns - train_columns

print('컬럼 목록 중 테스트 데이터에 없는 컬럼: ', missing_in_test)
print('컬럼 목록 중 훈련 데이터에 없는 컬럼: ', missing_in_train)

컬럼 목록 중 테스트 데이터에 없는 컬럼:  {'model_ A2', 'model_ S8', 'model_ RS7', 'model_ S5'}
컬럼 목록 중 훈련 데이터에 없는 컬럼:  set()


In [24]:
# 컬럼 목록 중 없는 컬럼의 값을 0으로 해서 채워줌
df_test_word['model_ RS7'] = 0
df_test_word['model_ S8'] = 0
df_test_word['model_ A2'] = 0
df_test_word['model_ S5'] = 0

In [25]:
# 데이터 결합
df_train = pd.concat([df_train_num, df_train_word], axis = 1)
df_test = pd.concat([df_test_num, df_test_word], axis = 1)
print(df_train)

          year   mileage       tax       mpg  engineSize  model_ A1  \
0     0.954545  0.020825  0.250000  0.127811    0.317460       True   
1     1.000000  0.014874  0.250000  0.056213    0.460317      False   
2     0.954545  0.002970  0.250000  0.124852    0.476190      False   
3     0.954545  0.030283  0.258621  0.111834    0.317460      False   
4     1.000000  0.029461  0.250000  0.189941    0.317460      False   
...        ...       ...       ...       ...         ...        ...   
7463  0.818182  0.134701  0.051724  0.234320    0.222222      False   
7464  0.863636  0.078987  0.250000  0.265680    0.253968      False   
7465  0.954545  0.029753  0.250000  0.127811    0.317460       True   
7466  0.954545  0.000054  0.250000  0.139053    0.317460      False   
7467  0.636364  0.542972  0.517241  0.117751    0.476190      False   

      model_ A2  model_ A3  model_ A4  model_ A5  ...  model_ S8  model_ SQ5  \
0         False      False      False      False  ...      False   

### 모델링

In [26]:
# 모델 생성
model = RandomForestRegressor(random_state = 77)

# 검증 데이터 생성
X_train, X_val, y_train_t, y_val = train_test_split(df_train.values, y_train['price'].values, test_size = 0.3)

# 모델 학습
model.fit(X_train, y_train_t)

# validation 데이터로 성능 평가
y_pred = model.predict(X_val)
print(np.sqrt(mean_squared_error(y_val, y_pred)))

2463.598683362138


In [27]:
# 예측
y_pred = model.predict(df_test)
df = pd.DataFrame(X_test['id'], columns = ['id'])
df['price'] = y_pred
print(df.head())

# csv파일 저장
df.to_csv('result1.csv', index = False)

     id      price
0  7469  33211.015
1  7470  18209.010
2  7471  35942.100
3  7472  17259.130
4  7473  16688.640


C:\Users\chunc\anaconda3\envs\pythonEx\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
